In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = train_transforms  # same for test


In [5]:
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("data/test", transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [9]:
class HairCNN(nn.Module):
    def __init__(self):
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2,2)
        # Calculate the size after conv + pool
        # Input: 200x200, kernel=3, padding=0, stride=1 => conv -> 198x198
        # After 2x2 pooling -> 99x99
        self.fc1 = nn.Linear(32*99*99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # no sigmoid here; use BCEWithLogitsLoss
        return x

model = HairCNN().to(device)

In [10]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [11]:
total_params = sum(p.numel() for p in model.parameters())
total_params

20073473

In [12]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [13]:
num_epochs = 10

train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train

    train_loss_list.append(epoch_loss)
    train_acc_list.append(epoch_acc)

    # validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    val_loss_list.append(val_epoch_loss)
    val_acc_list.append(val_epoch_acc)

    print(f"Epoch {epoch+1}/10  Train Acc={epoch_acc:.3f}  Loss={epoch_loss:.3f}  "
          f"Val Acc={val_epoch_acc:.3f}  Val Loss={val_epoch_loss:.3f}")

Epoch 1/10  Train Acc=0.626  Loss=0.665  Val Acc=0.662  Val Loss=0.610
Epoch 2/10  Train Acc=0.709  Loss=0.548  Val Acc=0.647  Val Loss=0.630
Epoch 3/10  Train Acc=0.764  Loss=0.486  Val Acc=0.597  Val Loss=0.699
Epoch 4/10  Train Acc=0.765  Loss=0.481  Val Acc=0.701  Val Loss=0.610
Epoch 5/10  Train Acc=0.786  Loss=0.440  Val Acc=0.652  Val Loss=0.621
Epoch 6/10  Train Acc=0.850  Loss=0.349  Val Acc=0.672  Val Loss=0.645
Epoch 7/10  Train Acc=0.877  Loss=0.301  Val Acc=0.682  Val Loss=0.695
Epoch 8/10  Train Acc=0.894  Loss=0.275  Val Acc=0.721  Val Loss=0.637
Epoch 9/10  Train Acc=0.921  Loss=0.211  Val Acc=0.711  Val Loss=0.685
Epoch 10/10  Train Acc=0.956  Loss=0.140  Val Acc=0.701  Val Loss=0.812


In [15]:
median_train_acc = np.median(train_acc_list)
median_train_acc

np.float64(0.818125)

In [18]:
std_train_loss = torch.tensor(train_loss_list).std(unbiased=False).item()
std_train_loss

0.15435850620269775

In [19]:
train_aug_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = datasets.ImageFolder("data/train", transform=train_aug_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)


In [20]:
aug_val_losses = []
aug_val_accs = []

for epoch in range(10):

    model.train()
    running_loss = 0.0

    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    aug_val_losses.append(val_epoch_loss)
    aug_val_accs.append(val_epoch_acc)

    print(f"[AUG] Epoch {epoch+1}/10  Val Acc={val_epoch_acc:.3f}  Val Loss={val_epoch_loss:.3f}")


[AUG] Epoch 1/10  Val Acc=0.627  Val Loss=0.751
[AUG] Epoch 2/10  Val Acc=0.697  Val Loss=0.570
[AUG] Epoch 3/10  Val Acc=0.697  Val Loss=0.592
[AUG] Epoch 4/10  Val Acc=0.706  Val Loss=0.556
[AUG] Epoch 5/10  Val Acc=0.677  Val Loss=0.581
[AUG] Epoch 6/10  Val Acc=0.716  Val Loss=0.511
[AUG] Epoch 7/10  Val Acc=0.716  Val Loss=0.569
[AUG] Epoch 8/10  Val Acc=0.682  Val Loss=0.594
[AUG] Epoch 9/10  Val Acc=0.652  Val Loss=0.692
[AUG] Epoch 10/10  Val Acc=0.692  Val Loss=0.549


In [21]:
mean_aug_val_loss = sum(aug_val_losses) / len(aug_val_losses)
mean_aug_val_loss

0.596602615625111

In [22]:
avg_last5_acc = sum(aug_val_accs[-5:]) / 5
avg_last5_acc

0.691542288557214